In [2]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import Counter
import re
from textblob import TextBlob
import arabic_reshaper
from bidi.algorithm import get_display
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
dataset = [
    {
    "question": "ما هي الأنواع المختلفة لنماذج التدريب في التعلم الآلي؟",
    "answer": "في مجال التعلم الآلي، توجد ثلاثة أنواع رئيسية لنماذج التدريب: التعلم الخاضع للإشراف (Supervised Learning)، حيث يتم استخدام بيانات مُعلّمة لتوجيه النموذج؛ التعلم غير الخاضع للإشراف (Unsupervised Learning)، والذي يعتمد على اكتشاف الأنماط والعلاقات داخل البيانات غير المُعلّمة؛ والتعلم التعزيزي (Reinforcement Learning)، حيث يتعلم النموذج من خلال التفاعل مع البيئة وتعزيز السلوك الذي يؤدي إلى تحقيق الأهداف.",
    "domain_terms": ["Supervised Learning", "Unsupervised Learning", "Reinforcement Learning", "التعلم الخاضع للإشراف", "التعلم غير الخاضع للإشراف", "التعلم التعزيزي"]
}
,
    {
    "question": "ما هو الفرق الرئيسي بين التعلم تحت الإشراف والتعلم غير المنظم؟",
    "answer": "الفرق الرئيسي بين التعلم تحت الإشراف (Supervised Learning) والتعلم غير المنظم (Unsupervised Learning) يكمن في وجود أو غياب البيانات المُعلّمة. التعلم تحت الإشراف يستخدم بيانات تحتوي على مخرجات أو إجابات صحيحة لتدريب النموذج، بينما التعلم غير المنظم يعتمد على بيانات غير معلّمة لا تحتوي على مخرجات محددة. الهدف في التعلم تحت الإشراف هو بناء نموذج للتنبؤ بالمخرجات بناءً على بيانات جديدة، في حين أن الهدف في التعلم غير المنظم هو اكتشاف الأنماط والعلاقات داخل البيانات.",
    "domain_terms": ["Supervised Learning", "Unsupervised Learning", "التعلم تحت الإشراف", "التعلم غير المنظم"]
}
,
    {
    "question": "ما هو الفرق بين السببية والارتباط؟",
    "answer": "السببية تشير إلى العلاقة التي تؤدي فيها التغييرات في أحد المتغيرات إلى تغييرات في متغير آخر، حيث تكون هناك علاقة بين السبب والنتيجة. أما الارتباط فيشير إلى العلاقة بين متغيرين أو أكثر من حيث كيفية تحركهم معًا، ولكن لا يعني بالضرورة أن أحدهم يسبب الآخر. على سبيل المثال، يمكن أن يكون هناك ارتباط بين درجات الحرارة العالية وزيادة في استهلاك المثلجات، لكن هذا لا يعني أن زيادة الحرارة هي السبب في زيادة الاستهلاك.",
    "domain_terms": ["السببية", "الارتباط", "العلاقة السببية", "الارتباط بين المتغيرات"]
}
]

# Simulated model-generated answers
generated_answers = ["""
    "
نماذج التدريب المستقبلية (Supervised Learning) : هي نوع من النماذج التي تتطلب معالجة بيانات مُحددة أو بيانات مُعلقة لتكون قادرة على تعلم خطوات تصحيح أو تحديد الخاصية.
نماذج التدريب الغير مستقبلية (Unsupervised Learning) : هي نوع من النماذج التي تتطلب معالجة بيانات غير مُحددة أو بيانات مُجمدة، لكي تكون قادرة على التعلم بشكل خاص في معرفة خاصية جديدة أو فرز البيانات في مجموعات غير مُحددة.
نماذج التدريب السياسية (Semi-Supervised Learning) : هي نوع من النماذج التي تتطلب معالجة بيانات مُحددة وغير مُحددة، لكي تكون قادرة على التعلم بشكل خاص في معرفة خاصية جديدة.
نماذج التدريب المستمر (Reinforcement Learning) : هي نوع من النماذج التي تتطلب معالجة بيانات غير مُحددة أو مُحددة، لكي تكون قادرة على التعلم بشكل خاص في إجراء الاختيارات المميزة في مستوى أعلى. """" ",

    """أحد الفرق الرئيسي الرئيسي بين التعلم تحت الإشراف (Supervised Learning) وغير المنظم (Unsupervised Learning) هو عدم وجود مثال معلم (Labeled Data) في الخطوة الثانية.

1. التعلم المنظم (Supervised Learning): يتم تدريب النموذج على مجموعة من البيانات المُعلمة (Labeled Data), حيث يكون هناك أسئلة والتي يتم عرضها للنموذج ويتم عرض إجابة صحيحة (Correct Answer) لها. بعد ذلك، يتم استخدام النموذج المُدرب للتعليم على مجموعة من البيانات التي لم تتم عرضها للنموذج أثناء التدريب (Test Data) ويتم قياس النجاح الرأسي (Accuracy) للنموذج.
2. التعلم الغير منظم (Unsupervised Learning): لا يوجد هناك إجابة صحيحة لأسئلة النموذج. النموذج يتعلم بالنفس من البيانات، ويقوم بفرزها إلى فئات أو تحديث خصائصها.

كلما زادت عدم الإشراف على البيانات، كلما زادت الصعوبة في التعليق على مشاكل حقيقية. بالطبيعة، إذا كانت المشكلة غير مُعرفة أو لا يوجد بيانات مُعلمة لها، فهناك حاجة إلى التعلم الغير منظم. في الخطوة الثانية، يتم استخدام النموذج المُدرب للتعليق على مشاكل حقيقية غير مُعرفة.""",
                     """
                     نعم، إن السببية والارتباط محدودتان للأسئلة في التعلم الآلي.

* **السببية** (features) هي المقاييس التي يتم تحديدها للمشكلة ويتم استخدامها في عملية التحليل. مثلًا، إذا كنت تريد تحليل أنواع سبائك جديدة، فتحدد مقاييسها المتعددة مثل الحجم النوعي، والرصيف، والمعدن الخام.
* **الارتباط** (relationship) هو علاقة بين السببية والإرجاع (output) مثل علاقة الحجم النوعي بالسبائك والتصنيع.

في التعلم الآلي، نحدد السببية ونحللها بأشكال مختلفة مثل الإحصاء الأساسي (regression) والإقرار (classification). على الإطلاق، يؤدي العمل على السببية إلى معرفة علاقتها بالإرجاع.
"""]

class ArabicTextEvaluator:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        """
        Initialize the Arabic text evaluator with necessary models and tools.

        Args:
            model_name: Name of the sentence transformer model to use
        """
        self.embedding_model = SentenceTransformer(model_name)
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method4

    def preprocess_arabic_text(self, text: str) -> str:
        """
        Preprocess Arabic text for better evaluation.

        Args:
            text: Input Arabic text

        Returns:
            Preprocessed text
        """
        # Normalize Unicode representations
        text = normalize_unicode(text)

        # Remove diacritics and tatweel
        text = re.sub(r'[\u064B-\u065F\u0640]', '', text)

        # Normalize Arabic characters
        replacements = {
            'أ': 'ا', 'إ': 'ا', 'آ': 'ا',
            'ى': 'ي', 'ة': 'ه'
        }
        for old, new in replacements.items():
            text = text.replace(old, new)

        return text

    def compute_semantic_coherence(self, generated: str, reference: str) -> float:
        """
        Compute semantic coherence between generated and reference texts.

        Args:
            generated: Generated Arabic text
            reference: Reference Arabic text

        Returns:
            Semantic coherence score
        """
        gen_emb = self.embedding_model.encode([generated], convert_to_tensor=True)
        ref_emb = self.embedding_model.encode([reference], convert_to_tensor=True)
        return float(cosine_similarity(gen_emb, ref_emb)[0][0])

    def compute_domain_coverage(
        self,
        text: str,
        domain_terms: List[str],
        weights: Dict[str, float] = None
    ) -> float:
        """
        Compute domain-specific terminology coverage with term importance weighting.

        Args:
            text: Input text
            domain_terms: List of domain-specific terms
            weights: Optional term importance weights

        Returns:
            Domain coverage score
        """
        if weights is None:
            weights = {term: 1.0 for term in domain_terms}

        text_tokens = set(simple_word_tokenize(text))
        covered_terms = text_tokens.intersection(set(domain_terms))

        weighted_coverage = sum(weights.get(term, 1.0) for term in covered_terms)
        total_weight = sum(weights.values())

        return weighted_coverage / total_weight if total_weight > 0 else 0.0

    def compute_enhanced_wssa(
        self,
        generated_answers: List[str],
        correct_answers: List[str],
        domain_terms: List[List[str]],
        weights: Dict[str, float] = None
    ) -> List[Dict[str, float]]:
        """
        Compute Enhanced WSSA (Weighted Semantic Similarity with Arabic-specific Adjustments)

        Args:
            generated_answers: List of generated answers
            correct_answers: List of reference answers
            domain_terms: List of domain-specific terms for each answer
            weights: Optional component weights

        Returns:
            List of evaluation metrics for each answer
        """
        if weights is None:
            weights = {
                'semantic': 0.35,
                'domain': 0.25,
                'fluency': 0.15,
                'bleu': 0.15,
                'rouge': 0.10
            }

        results = []

        for gen_ans, corr_ans, terms in zip(generated_answers, correct_answers, domain_terms):
            # Preprocess texts
            gen_ans_proc = self.preprocess_arabic_text(gen_ans)
            corr_ans_proc = self.preprocess_arabic_text(corr_ans)

            # Component scores
            semantic_score = self.compute_semantic_coherence(gen_ans_proc, corr_ans_proc)
            domain_score = self.compute_domain_coverage(gen_ans_proc, terms)

            # BLEU score with Arabic-specific tokenization
            bleu_score = sentence_bleu(
                [simple_word_tokenize(corr_ans_proc)],
                simple_word_tokenize(gen_ans_proc),
                smoothing_function=self.smoothing
            )

            # ROUGE scores
            rouge_scores = self.rouge_scorer.score(corr_ans_proc, gen_ans_proc)
            rouge_l = rouge_scores['rougeL'].fmeasure

            # Compute weighted final score
            final_score = (
                weights['semantic'] * semantic_score +
                weights['domain'] * domain_score +
                weights['bleu'] * bleu_score +
                weights['rouge'] * rouge_l
            )

            metrics = {
                'final_score': final_score,
                'semantic_coherence': semantic_score,
                'domain_coverage': domain_score,
                'bleu': bleu_score,
                'rouge_l': rouge_l
            }

            results.append(metrics)

        return results
correct_answers = [item["answer"] for item in dataset]
domain_terms = [item["domain_terms"] for item in dataset]
# Example usage
evaluator = ArabicTextEvaluator()
results = evaluator.compute_enhanced_wssa(generated_answers, correct_answers, domain_terms)

# Display results
for i, metrics in enumerate(results, 1):
    print(f"\nQuestion {i} Evaluation Metrics:")
    for metric, score in metrics.items():
        print(f"{metric}: {score:.4f}")


Question 1 Evaluation Metrics:
final_score: 0.3978
semantic_coherence: 0.8830
domain_coverage: 0.0000
bleu: 0.0580
rouge_l: 0.8000

Question 2 Evaluation Metrics:
final_score: 0.3472
semantic_coherence: 0.8573
domain_coverage: 0.0000
bleu: 0.0826
rouge_l: 0.3478

Question 3 Evaluation Metrics:
final_score: 0.3888
semantic_coherence: 0.9265
domain_coverage: 0.2500
bleu: 0.0132
rouge_l: 0.0000
